In [21]:
# Import libraries
import numpy as np
import os , csv
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd


In [14]:
# Load desired data from 1 session 1 animal
# Note that there are 340 trials in 1 session
# (For more info https://github.com/nsteinme/steinmetz-et-al-2019/wiki/data-files)

data_path= '/Users/xinweichia/Documents/connected_lizards/Steinmetz_dataset/Richards_2017-10-31'

os.chdir(data_path) # Change working directory


trials_intervals = np.load('trials.intervals.npy') # in seconds
spike_times = np.load('spikes.times.npy') * 1000 # Unbinned spike times in ms
trials_gocue_times = np.load('trials.goCue_times.npy') 
trials_response_choice = np.load('trials.response_choice.npy') # -1 left, 1, right, 0 no response
spontaneous_intervals = np.load('spontaneous.intervals.npy')
trials_response_time = np.load('trials.response_times.npy')
spike_clusters = np.load('spikes.clusters.npy')
site_positions = np.load('channels.sitePositions.npy')
clusters_depths = np.load('clusters.depths.npy')
clusters_annotation = np.load('clusters._phy_annotation.npy')
channel_sites = np.load('channels.site.npy')
channels_brainlocation = pd.read_csv('channels.brainLocation.tsv', sep='\t')
clusters_probes = np.load('clusters.probes.npy')
channels_probe = np.load('channels.probe.npy')


In [36]:
# Taken from https://github.com/MouseLand/steinmetz2019_NMA/blob/master/steinmetz_loader.py
# To obtain brain regions
def get_good_cells(fdirpath): #
    # location in brain of each neuron
    brain_loc = os.path.join(fdirpath, "channels.brainLocation.tsv")

    good_cells = (np.load(os.path.join(fdirpath, "clusters._phy_annotation.npy")) >= 2 ).flatten()
    clust_channel = np.load(os.path.join(fdirpath, "clusters.peakChannel.npy")).astype(int) - 1
    br = []
    with open(brain_loc, 'r') as tsv:
        tsvin = csv.reader(tsv, delimiter="\t")
        k=0
        for row in tsvin:
            if k>0:
                br.append(row[-1])
            k+=1
    br = np.array(br)
    good_cells = np.logical_and(good_cells, clust_channel.flatten()<len(br))
    brain_region = br[clust_channel[:,0]]


    return good_cells, brain_region, br

good_cells, brain_regions ,br = get_good_cells('')

In [5]:
# Initialising spike time dataset

# Using clusters._phy_annotation.npy obtain valid clusters (i.e. >= 2)
valid_clusters_idx = np.array(np.where(clusters_annotation>=2))[0]

spike_time_cells = np.empty(len(valid_clusters_idx), dtype=object) # Initalise empty object
for i,cell_idx in enumerate(valid_clusters_idx):
  # Create a spike time arrays, where each array in the array is a spike time of a cell
  spike_time_cells[i] = spike_times[(np.where(spike_clusters == cell_idx)[0])]

# Bin spike times into 10ms intervals
bin_size = 10

spike_time_binned = np.empty(len(valid_clusters_idx), dtype=object) # Initalise empty object
sum_spikes = np.empty(len(valid_clusters_idx), dtype=object) # Initalise empty object

for cell_num in np.arange(len(spike_time_cells)): 
  spike_time_hist = np.histogram(spike_time_cells[cell_num],bins = np.arange(0,np.floor(spike_time_cells[cell_num][-1]),bin_size))
  spike_time_binned[cell_num] = spike_time_hist[0]
  sum_spikes[cell_num] = np.sum(spike_time_binned[cell_num])


cell_spikes_max = np.argmax(sum_spikes) # cell with the maximum number of spikes for plotting purposes

In [ ]:
# Sort cells into trial types and relevant epoch

# Get response choice trials types
right_choice_trials = np.where(trials_response_choice == -1)[0]
left_choice_trials = np.where(trials_response_choice == 1)[0]
no_response_choice_trials = np.where(trials_response_choice == 0)[0]

response_epoch_duration  = 400 * 0.1 # 400ms in 10ms bins

# Sort into trials
spike_time_binned_trial = np.empty(len(spike_time_cells), dtype=object)
spike_time_binned_trial_response = np.empty(len(spike_time_cells), dtype=object)
for cell_num in np.arange(len(spike_time_cells)):
  spike_time_binned_trial[cell_num] = np.empty(len(trials_intervals), dtype=object)
  spike_time_binned_trial_response[cell_num] = np.empty(len(trials_intervals), dtype=object)
  for i,trials_start_end in enumerate(trials_intervals):
    # Sort spikes into their trial numbers. 
    spike_time_binned_trial[cell_num][i] = spike_time_binned[cell_num][ int(np.floor(trials_start_end[0]*100)) : int(np.floor(trials_start_end[1]*100))]
    # Using Go_cue time to splice a trial into gocue onset : gocue onset +400ms
    spike_time_binned_trial_response[cell_num][i] = spike_time_binned[cell_num][(int(np.floor(trials_gocue_times[i]*100))) : (int(np.floor(trials_gocue_times[i]*100) + epoch_duration))]


# Sort trials into response type

left_response_spike_time_response_epoch = The 
for cell_num in np.arange(len(spike_time_cells)):